In [1]:
# This is a Python 3 environment

# Base level imports for data science work
import numpy as np 
import pandas as pd
import re,string,unicodedata
import os
from os import path
from json import decoder
import glob, os, json
import re

# Visualization Libs
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# NLP Libs
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
from keras.preprocessing import text, sequence
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

# Additional Libs
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet

# Deep Learning Libs
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

In [2]:
df = pd.read_csv("https://chruezfalsch.ch/df_FANG-COVID_cleaned_400-3500.csv")
del df['Unnamed: 0'] 

In [3]:
#!wget https://chruezfalsch.ch/ba_model_1.joblib
#!wget https://chruezfalsch.ch/ba_model_1_cv.joblib
#!wget https://chruezfalsch.ch/ba_model_1_tfidf.joblib

!wget https://chruezfalsch.ch/ba_model_3.joblib
!wget https://chruezfalsch.ch/ba_model_3_cv.joblib
!wget https://chruezfalsch.ch/ba_model_3_tfidf.joblib

--2022-05-25 19:36:21--  https://chruezfalsch.ch/ba_model_3.joblib
Resolving chruezfalsch.ch (chruezfalsch.ch)... 185.151.30.135, 2a07:7800::135
Connecting to chruezfalsch.ch (chruezfalsch.ch)|185.151.30.135|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36388093 (35M)
Saving to: ‘ba_model_3.joblib’

ba_model_3.joblib   100%[===================>]  34.70M  25.6MB/s    in 1.4s    

2022-05-25 19:36:23 (25.6 MB/s) - ‘ba_model_3.joblib’ saved [36388093/36388093]

--2022-05-25 19:36:23--  https://chruezfalsch.ch/ba_model_3_cv.joblib
Resolving chruezfalsch.ch (chruezfalsch.ch)... 185.151.30.135, 2a07:7800::135
Connecting to chruezfalsch.ch (chruezfalsch.ch)|185.151.30.135|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40034816 (38M)
Saving to: ‘ba_model_3_cv.joblib’

ba_model_3_cv.jobli 100%[===================>]  38.18M  10.1MB/s    in 5.1s    

2022-05-25 19:36:29 (7.54 MB/s) - ‘ba_model_3_cv.joblib’ saved [40034816/40034816]

--2022-05

In [4]:
import pickle
from joblib import dump, load
#mnb = load('ba_model_1.joblib')
#cv = pickle.load(open('ba_model_1_cv.joblib','rb'))
#tfidf = pickle.load(open('ba_model_1_tfidf.joblib','rb'))

In [5]:
mnb = load('ba_model_3.joblib')
cv = pickle.load(open('ba_model_3_cv.joblib','rb'))
tfidf = pickle.load(open('ba_model_3_tfidf.joblib','rb'))

#Model

In [6]:
X_test = df.text
y_test = df.fake

# Transform the test dataset
cv_test=cv.transform(X_test)
print('Bag of words Test:',cv_test.shape)

Bag of words Test: (28363, 1137107)


In [7]:
# Transform the test dataset
tfidf_test=tfidf.transform(X_test)
print('TF-IDF Test:',tfidf_test.shape)

TF-IDF Test: (28363, 1137107)


In [8]:
# Predicting the model for bag of words
mnb_cv_predict=mnb.predict(cv_test)

# Predicting the model for tf-idf features
mnb_tfidf_predict=mnb.predict(tfidf_test)

In [9]:
# Check the accuracy score for bag of words
mnb_cv_score=accuracy_score(y_test, mnb_cv_predict)
print("Naive Bayes Bag of words accuracy score:", mnb_cv_score)

# Check the accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(y_test,mnb_tfidf_predict)
print("Naive Bayes TF-IDF accuracy score:", mnb_tfidf_score)

Naive Bayes Bag of words accuracy score: 0.7837675845291401
Naive Bayes TF-IDF accuracy score: 0.7628600641681064


In [10]:
mnb_cv_report = classification_report(y_test, mnb_cv_predict, target_names = ['0','1'])
print(mnb_cv_report)
mnb_tfidf_report = classification_report(y_test, mnb_tfidf_predict, target_names = ['0','1'])
print(mnb_tfidf_report)

              precision    recall  f1-score   support

           0       0.84      0.87      0.86     20747
           1       0.61      0.54      0.57      7616

    accuracy                           0.78     28363
   macro avg       0.72      0.71      0.71     28363
weighted avg       0.78      0.78      0.78     28363

              precision    recall  f1-score   support

           0       0.86      0.80      0.83     20747
           1       0.55      0.65      0.60      7616

    accuracy                           0.76     28363
   macro avg       0.71      0.73      0.71     28363
weighted avg       0.78      0.76      0.77     28363



#Model (ausbalanciert)

In [11]:
dfb=df[df.fake == 0].sample(5000)
dfb = dfb.append(df[df.fake == 1].sample(5000))
X_test = dfb.text
y_test = dfb.fake

# Transform the test dataset
cv_test=cv.transform(X_test)
print('Bag of words Test:',cv_test.shape)

Bag of words Test: (10000, 1137107)


In [12]:
# Transform the test dataset
tfidf_test=tfidf.transform(X_test)
print('TF-IDF Test:',tfidf_test.shape)

TF-IDF Test: (10000, 1137107)


In [13]:
# Predicting the model for bag of words
mnb_cv_predict=mnb.predict(cv_test)

# Predicting the model for tf-idf features
mnb_tfidf_predict=mnb.predict(tfidf_test)

In [14]:
# Check the accuracy score for bag of words
mnb_cv_score=accuracy_score(y_test, mnb_cv_predict)
print("Naive Bayes Bag of words accuracy score:", mnb_cv_score)

# Check the accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(y_test,mnb_tfidf_predict)
print("Naive Bayes TF-IDF accuracy score:", mnb_tfidf_score)

Naive Bayes Bag of words accuracy score: 0.7089
Naive Bayes TF-IDF accuracy score: 0.7279


In [15]:
mnb_cv_report = classification_report(y_test, mnb_cv_predict, target_names = ['0','1'])
print(mnb_cv_report)
mnb_tfidf_report = classification_report(y_test, mnb_tfidf_predict, target_names = ['0','1'])
print(mnb_tfidf_report)

              precision    recall  f1-score   support

           0       0.66      0.87      0.75      5000
           1       0.81      0.54      0.65      5000

    accuracy                           0.71     10000
   macro avg       0.73      0.71      0.70     10000
weighted avg       0.73      0.71      0.70     10000

              precision    recall  f1-score   support

           0       0.70      0.80      0.75      5000
           1       0.77      0.65      0.71      5000

    accuracy                           0.73     10000
   macro avg       0.73      0.73      0.73     10000
weighted avg       0.73      0.73      0.73     10000

